In [2]:
%%pyspark
billing_silver = spark.read.format("delta").load("abfss://silverlayerdata@energyscotish.dfs.core.windows.net/billing_silver/")
silver_customer_data = spark.read.format("delta").load("abfss://silverlayerdata@energyscotish.dfs.core.windows.net/customer_silver/")
silver_region_data = spark.read.format("delta").load("abfss://silverlayerdata@energyscotish.dfs.core.windows.net/regions_data/")
silver_iot_data = spark.read.format("delta").load("abfss://silverlayerdata@energyscotish.dfs.core.windows.net/silver/iot_data/")
silver_installation_data = spark.read.format("delta").load("abfss://silverlayerdata@energyscotish.dfs.core.windows.net/silver_installation_data/")

StatementMeta(TestCluster, 14, 3, Finished, Available, Finished)

In [18]:
%%pyspark

# Create temporary views
billing_silver.createOrReplaceTempView("billing_silver_view")
silver_customer_data.createOrReplaceTempView("customer_silver_view")
silver_region_data.createOrReplaceTempView("region_silver_view")
silver_iot_data.createOrReplaceTempView("iot_silver_view")
silver_installation_data.createOrReplaceTempView("installation_silver_view")

StatementMeta(TestCluster, 14, 19, Finished, Available, Finished)

#creating Dim_Tables 

In [28]:

%%pyspark
gold_customer_dim = silver_customer_data.selectExpr(
    "CustomerID",
    "CustomerName AS CustomerName",  # Use the correct column name
    "ContactEmail AS ContactEmail",
    "ContactPhone AS ContactPhone",
    "TRIM(REPLACE(REPLACE(REPLACE(REPLACE(Address, 'St', 'Street'), 'Ln', 'Lane'), 'Ave', 'Avenue'), 'Blvd', 'Boulevard')) AS Address",
    "DateOfBirth",
    "RegionID",
    "CURRENT_TIMESTAMP() AS LastUpdated"
)


StatementMeta(TestCluster, 14, 29, Finished, Available, Finished)

In [29]:
%%pyspark
gold_region_dim = silver_region_data.selectExpr(
    "RegionID",
    "RegionName",
    "State",
    "ZipCode",
    "CURRENT_TIMESTAMP() AS LastUpdated"
).distinct()


StatementMeta(TestCluster, 14, 30, Finished, Available, Finished)

In [30]:
%%pyspark
gold_meter_dim = spark.sql("""
    SELECT DISTINCT
        MeterID,
        InstallationID,
        CustomerID,
        DataIngestionDate,
        CURRENT_TIMESTAMP() AS LastUpdated
    FROM installation_silver_view
""")


StatementMeta(TestCluster, 14, 31, Finished, Available, Finished)

In [31]:
%%pyspark
gold_date_dim = spark.sql("""
    SELECT DISTINCT
        Date AS DateKey,
        Date AS FullDate,
        YEAR(Date) AS Year,
        MONTH(Date) AS Month,
        DAY(Date) AS Day
    FROM silver_iot_data
""")



StatementMeta(TestCluster, 14, 32, Finished, Available, Finished)

In [32]:

%%pyspark
gold_billing_fact = spark.sql("""
    SELECT
        b.BillID,
        b.CustomerID,
        b.BillingMonth,
        b.TotalConsumption,
        b.TotalAmount,
        b.PaymentStatus,
        m.MeterID,
        c.RegionID,
        CURRENT_TIMESTAMP() AS LastUpdated
    FROM Billing_silver_view b
    JOIN customer_silver_view c ON b.CustomerID = c.CustomerID
    JOIN installation_silver_view m ON c.CustomerID = m.CustomerID
""")

 


StatementMeta(TestCluster, 14, 33, Finished, Available, Finished)

In [33]:
%%pyspark
gold_iot_fact = spark.sql("""
    SELECT
        i.MeterID,
        i.Timestamp,
        i.Date,
        i.ConsumptionKWH,
        i.DeviceStatus,
        i.AnomalyFlag,
        CURRENT_TIMESTAMP() AS LastUpdated
    FROM silver_iot_data i
""")



StatementMeta(TestCluster, 14, 34, Finished, Available, Finished)

In [34]:
%%pyspark
gold_region_dim.write.format("delta").mode("overwrite").save("abfss://goldayer@energyscotish.dfs.core.windows.net/region_dim")
gold_meter_dim.write.format("delta").mode("overwrite").save("abfss://goldayer@energyscotish.dfs.core.windows.net/meter_dim")
gold_date_dim.write.format("delta").mode("overwrite").save("abfss://goldayer@energyscotish.dfs.core.windows.net/date_dim") what about fact 

StatementMeta(TestCluster, 14, 35, Finished, Available, Finished)

In [35]:
%%pyspark

gold_billing_fact.write.format("delta").mode("overwrite").save("abfss://goldayer@energyscotish.dfs.core.windows.net/billing_fact")

gold_iot_fact.write.format("delta").mode("overwrite").save("abfss://goldayer@energyscotish.dfs.core.windows.net/iot_fact")


StatementMeta(TestCluster, 14, 36, Finished, Available, Finished)